In [1]:
import pandas as pd
import numpy as np
#import win32com.client as win32
import pytesseract
import matplotlib.pyplot as plt
import nltk
import cv2
import os
import json
import torch
#from unidecode import unidecode

In [2]:
os.chdir(r'C:\Users\puboggavarapu\Desktop\skills&studies\kaggle\covid_19')

### Getting full text for all files that have full text available

In [4]:
metadata = pd.read_csv('metadata.csv')

In [5]:
metadata['full_text_file'].value_counts()

custom_license        20873
comm_use_subset        8803
noncomm_use_subset     2133
biorxiv_medrxiv        1020
Name: full_text_file, dtype: int64

### Example

In [7]:
#Example
path = './' + metadata.loc[3,'full_text_file'] + '/' + metadata.loc[3,'full_text_file'] + '/' + metadata.loc[3,'sha'].split(';')[0] + '.json'
[text['text'] for text in json.loads(open(path).read())['abstract']]

['Middle-aged female identical twins, one of whom had systemic lupus erythematosus (SLE), were evaluated for immunologic reactivity to previous antigenic challenges, including primary immunization with a foreign antigen, keyhole limpet hemocyanin (KLH). These two women had lived together for all of their 58 years and neither was receiving anti-inflammatory or immunosuppressive drugs at the time of these studies. Both twins demonstrated comparable 7s and 19s humoral antibody response to KLH, as well as similar viral antibody titers. However, the twin with SLE was anergic to common antigens, streptokinase-streptodornase, Trichophyton and Candida; furthermore delayed hypersensitivity to KLH did not develop after immunization. This observed discrepancy between humoral and cellular immunity in genetically similar subjects may be significant in the pathogenesis of SLE.',
 'Reports of an increased incidence of systemic lupus erythematosus (SLE), other connective tissue diseases, and serologic

In [8]:
metadata_with_file = metadata[metadata.sha.notnull()]
metadata_with_file.shape

(28462, 15)

In [9]:
metadata_without_file = metadata[~metadata.sha.notnull()]
metadata_without_file.shape

(15758, 15)

In [10]:
#For files which have json object - 
#    if full_text exists:
#       take full text
#    else
#       take abstract
texts_with_file = metadata_with_file.apply(lambda x: [text['text'] for text in json.loads(open(('./' + x['full_text_file'] + '/' + x['full_text_file'] + '/' + x['sha'].split(';')[0] + '.json')).read())['abstract']] if x['has_full_text']==False else [text['text'] for text in json.loads(open('./' + x['full_text_file'] + '/' + x['full_text_file'] + '/' + x['sha'].split(';')[0] + '.json').read())['body_text']] ,axis=1)

In [11]:
# Get paragraphs out of file-wise texts
paras = [para for paras in texts_with_file for para in paras]

In [ ]:
# Getting memory error unfortunately when trying to convert paragraph into sentences
from textblob import TextBlob
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [199]:
model_file = './infersent/infersent1.pkl'

In [201]:
from models import InferSent

In [203]:
MODEL_PATH = './infersent/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [204]:
W2V_PATH = 'glove/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [ ]:
%%time
#build vocab

#infersent.build_vocab(sentences, tokenize=True)
infersent.build_vocab(paras, tokenize=True)

In [ ]:
dict_embeddings = {}
for i in range(len(sentences)):
    print(i)
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [219]:
# embed the paragraphs(ideally should be sentences)
embeddings = infersent.encode(paras, tokenize=True)

In [18]:
#Pre process fucntion
import re
import nltk
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

import enchant
d = enchant.Dict("en_US")

nltk.download('words')
words = set(nltk.corpus.words.words())

def preprocess(data):
    print(1)
    #lower case
    text_lower = data['Text'].apply(lambda x : str(x).lower())
    data['Text'] = text_lower
    #remove non english terms
    print(2)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(w for w in str(x).split() if d.check(w.lower())))
    
    #remove numbers and special characters
    print(3)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: x.replace(r"[^a-zA-Z ]+", " ").strip())
    
    data.loc[:,'Text'] = data['Text'].apply(lambda x:x.translate(str.maketrans('', '', string.punctuation)))
    
    data.loc[:,'Text'] = data['Text'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
    
    
    
    #remove punctuation
    #data.loc[:,'Text'] = data['Text'].str.replace('[^\w\s]','')
    
    #remove sentences with less than or equal to three words
    print(4)
    def remove_less_than_3_word_sentences(data):
        count = data['Text'].str.split().str.len()
        data = data[(count>3)]
        return data
    data = remove_less_than_3_word_sentences(data)
    
    #remove stop words
    print(5)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(item for item in w_tokenizer.tokenize(x) if item not in stop))

    
    #lemmatize
    print(6)
    data.loc[:,'Text'] = data['Text'].apply(lambda x: " ".join(lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(str(x))))
    
    return data

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\puboggavarapu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\puboggavarapu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## CAUTION

In [ ]:
# #save changes to git
# import os
# from subprocess import check_call
# from shlex import split

# def post_save( os_path):
#     """post-save hook for doing a git commit / push"""
#     check_call(split('git add {}'.format(filename)), cwd=workdir)
#     check_call(split('git commit -m "notebook save" {}'.format(filename)), cwd=workdir)
#     check_call(split('git push'), cwd=workdir)
    
    
# file_full_path = r'C:\Users\puboggavarapu\Desktop\skills&studies\kaggle\github\covid19\Baseline_Model.ipynb'
# post_save(file_full_path)